In [1]:
from typing import List, Tuple

In [2]:
! pip install transformers datasets torch torchvision torchaudio evaluate scikit-learn numpy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.2 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset, Dataset
import evaluate
import numpy as np
from sklearn.metrics import classification_report
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from transformers import pipeline


In [4]:
import os
import json
from pathlib import Path
from transformers import AutoModelForCausalLM
import random
random.seed(42)
torch.manual_seed(42)

# Pipeline дообучения модели

In [5]:
SEED = 42

## Загрузим токенайзер

In [6]:
MODEL_NAME = 'DeepPavlov/rubert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [7]:
seq = 'Привет о дивный новый мир!'
print(tokenizer.encode(seq))
print(tokenizer.convert_ids_to_tokens(tokenizer.encode(seq)))

[101, 77527, 612, 32136, 1916, 10303, 6913, 106, 102]
['[CLS]', 'Привет', 'о', 'див', '##ный', 'новый', 'мир', '!', '[SEP]']


## Загрузим данные

In [8]:
DATASET_NAME = 'Davlan/sib200'
DATASET_LANGUAGE = 'rus_Cyrl'
train_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='train')
validation_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='validation')
test_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='test')

README.md: 0.00B [00:00, ?B/s]

train.tsv: 0.00B [00:00, ?B/s]

dev.tsv: 0.00B [00:00, ?B/s]

test.tsv: 0.00B [00:00, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [9]:
from collections import Counter

def print_class_distribution(dataset, split_name: str):
    categories = dataset['category']
    counter = Counter(categories)
    print(f"\n{split_name} set class distribution:")
    for cls in sorted(counter):
        print(f"  {cls}: {counter[cls]} examples")

print_class_distribution(train_set, "Train")
print_class_distribution(validation_set, "Validation")
print_class_distribution(test_set, "Test")


Train set class distribution:
  entertainment: 65 examples
  geography: 58 examples
  health: 77 examples
  politics: 102 examples
  science/technology: 176 examples
  sports: 85 examples
  travel: 138 examples

Validation set class distribution:
  entertainment: 9 examples
  geography: 8 examples
  health: 11 examples
  politics: 14 examples
  science/technology: 25 examples
  sports: 12 examples
  travel: 20 examples

Test set class distribution:
  entertainment: 19 examples
  geography: 17 examples
  health: 22 examples
  politics: 30 examples
  science/technology: 51 examples
  sports: 25 examples
  travel: 40 examples


LEEET'S BALANCE

In [10]:
from collections import Counter

train_categories = train_set['category']
counter = Counter(train_categories)
print("Текущее распределение:")
for cls in sorted(counter):
    print(f"  {cls}: {counter[cls]}")

target_size = 120

to_generate = {cls: max(0, target_size - count) for cls, count in counter.items()}
print("\надо сгенерировать:")
for cls, n in to_generate.items():
    if n > 0:
        print(f"  {cls}: {n}")

Текущее распределение:
  entertainment: 65
  geography: 58
  health: 77
  politics: 102
  science/technology: 176
  sports: 85
  travel: 138
\надо сгенерировать:
  geography: 62
  entertainment: 55
  politics: 18
  health: 43
  sports: 35


In [11]:
QWEN_MODEL_NAME = "Qwen/Qwen1.5-1.8B-Chat"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}")

qwen_tokenizer = AutoTokenizer.from_pretrained(QWEN_MODEL_NAME, trust_remote_code=True)
qwen_model = AutoModelForCausalLM.from_pretrained(
    QWEN_MODEL_NAME,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto",
    trust_remote_code=True
)
qwen_model.eval()

qwen_pipe = pipeline(
    "text-generation",
    model=qwen_model,
    tokenizer=qwen_tokenizer,
    pad_token_id=qwen_tokenizer.eos_token_id
)

cuda


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

Device set to use cuda:0


In [12]:
PROMPT_TEMPLATES = {
    "entertainment": "Напиши короткий новостной текст (2–4 предложения) на русском языке в категории «развлечения». Тема: кино, музыка или знаменитости. Стиль: информативный, нейтральный.",
    "geography": "Напиши короткий новостной текст (2–4 предложения) на русском языке в категории «география». Тема: интересные места, регионы, природные объекты. Стиль: познавательный.",
    "health": "Напиши короткий новостной текст (2–4 предложения) на русском языке в категории «здоровье». Тема: профилактика, питание, медицина. Стиль: нейтральный, фактологический.",
    "politics": "Напиши короткий новостной текст (2–4 предложения) на русском языке в категории «политика». Тема: выборы, законодательство или международные отношения. Стиль: объективный.",
    "science/technology": "Напиши короткий новостной текст (2–4 предложения) на русском языке в категории «наука и технологии». Тема: ИИ, космос, гаджеты или открытия. Стиль: информативный.",
    "sports": "Напиши короткий новостной текст (2–4 предложения) на русском языке в категории «спорт». Тема: футбол, Олимпиада, соревнования. Стиль: динамичный, но нейтральный.",
    "travel": "Напиши короткий новостной текст (2–4 предложения) на русском языке в категории «путешествия». Тема: туристические маршруты, отели, советы. Стиль: познавательный."
}

промпты очев. не сам писал

In [13]:
synthetic_texts = []
synthetic_labels = []

for category, n_needed in to_generate.items():
    if n_needed == 0:
        continue
    print(f"\nГенерация {n_needed} текстов для категории: {category}")
    prompt = PROMPT_TEMPLATES.get(category, f"Напиши короткий текст на русском языке по теме: {category}.")

    for i in range(n_needed):
        try:
            generated = qwen_pipe(
                prompt,
                max_new_tokens=100,
                do_sample=True,
                temperature=0.9,
                top_p=0.95,
                num_return_sequences=1
            )
            text = generated[0]['generated_text'].strip()
            if text.startswith(prompt):
                text = text[len(prompt):].strip()
            if text:
                synthetic_texts.append(text)
                synthetic_labels.append(category)
        except Exception as e:
            print(f"Ошибка при генерации для {category}: {e}")
            continue



Генерация 62 текстов для категории: geography


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



Генерация 55 текстов для категории: entertainment

Генерация 18 текстов для категории: politics

Генерация 43 текстов для категории: health

Генерация 35 текстов для категории: sports


In [16]:
with open("synthetic_sib200.json", "w", encoding="utf-8") as f:
    json.dump(
        [{"text": t, "category": c} for t, c in zip(synthetic_texts, synthetic_labels)],
        f,
        ensure_ascii=False,
        indent=2
    )
# with open("synthetic_sib200.json", "r", encoding="utf-8") as f:
#     syn_data = json.load(f)
#     synthetic_texts = [x["text"] for x in syn_data]
#     synthetic_labels = [x["category"] for x in syn_data]

LEEET'S MERGE

In [17]:
combined_texts = list(train_set['text']) + synthetic_texts
combined_categories = list(train_set['category']) + synthetic_labels

balanced_train_set = Dataset.from_dict({
    'text': combined_texts,
    'category': combined_categories
})

print("Новое распределение:")
new_counter = Counter(combined_categories)
for cls in sorted(new_counter):
    print(f"  {cls}: {new_counter[cls]}")

Новое распределение:
  entertainment: 120
  geography: 120
  health: 120
  politics: 120
  science/technology: 176
  sports: 120
  travel: 138


In [19]:
print(train_set)

Dataset({
    features: ['index_id', 'category', 'text'],
    num_rows: 701
})


In [ ]:
torch.cuda.empty_cache()

## Токенизируем тексты

In [20]:
MINIBATCH_SIZE = 32
tokenized_train_set = balanced_train_set.map(
    lambda it: tokenizer(it['text'], truncation=True, max_length=128),
    batched=True, batch_size=MINIBATCH_SIZE
)
tokenized_validation_set = validation_set.map(
    lambda it: tokenizer(it['text'], truncation=True, max_length=128),
    batched=True, batch_size=MINIBATCH_SIZE
)

Map:   0%|          | 0/914 [00:00<?, ? examples/s]

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

In [21]:
print(tokenized_train_set)

Dataset({
    features: ['text', 'category', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 914
})


## Загрузим метрику для оценки качества модели на задаче классификации

In [22]:
cls_metric = evaluate.load('f1')

In [23]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return cls_metric.compute(predictions=predictions, references=labels, average='macro')

## Создадим вспомогательный объект для трансформации данных в тензор

In [24]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Конвертируем категории в целевые метки

In [27]:
list_of_categories = sorted(list(
    set(balanced_train_set['category']) | set(validation_set['category']) | set(test_set['category'])
))
indices_of_categories = list(range(len(list_of_categories)))
n_categories = len(list_of_categories)
print(f'Categories for classification are: {list_of_categories}')
id2label = dict(zip(indices_of_categories, list_of_categories))
label2id = dict(zip(list_of_categories, indices_of_categories))

Categories for classification are: ['entertainment', 'geography', 'health', 'politics', 'science/technology', 'sports', 'travel']


In [28]:
labeled_train_set = tokenized_train_set.add_column(
    'label',
    [label2id[val] for val in tokenized_train_set['category']]
)
labeled_validation_set = tokenized_validation_set.add_column(
    'label',
    [label2id[val] for val in tokenized_validation_set['category']]
)

In [29]:
print(labeled_train_set)

Dataset({
    features: ['text', 'category', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
    num_rows: 914
})


# TensorBoard

## Загружаем наш BERT
И определем его на видеокарту

In [30]:
classifier = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=n_categories, id2label=id2label, label2id=label2id
)
for param in classifier.parameters(): param.data = param.data.contiguous()

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Этап обучения

In [48]:
training_args = TrainingArguments(
    output_dir='rubert_sib200',
    learning_rate=2e-5,
    per_device_train_batch_size=MINIBATCH_SIZE,
    per_device_eval_batch_size=MINIBATCH_SIZE,
    num_train_epochs=8,
    weight_decay=1e-3,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    logging_steps = 5,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    # fp16=(device.type != 'cpu'),
    data_seed=SEED,
    # label_smoothing_factor=0.05
)

In [49]:
trainer = Trainer(
    model=classifier,
    args=training_args,
    train_dataset=labeled_train_set,
    eval_dataset=labeled_validation_set,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-1253556369.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [50]:
import wandb
wandb.init(mode="disabled")

In [51]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,0.004500,0.847496,0.846084
2,0.002100,0.975436,0.870079
3,0.001300,0.986889,0.850705
4,0.001000,1.034082,0.842919
5,0.000900,1.033078,0.838172
6,0.000800,0.953720,0.869856
7,0.000700,0.955816,0.869856
8,0.000700,0.950788,0.869856


TrainOutput(global_step=232, training_loss=0.004920378936542969, metrics={'train_runtime': 461.0832, 'train_samples_per_second': 15.858, 'train_steps_per_second': 0.503, 'total_flos': 402832887093600.0, 'train_loss': 0.004920378936542969, 'epoch': 8.0})

In [52]:
trainer.evaluate()

{'eval_loss': 0.847495973110199,
 'eval_f1': 0.8460842835157779,
 'eval_runtime': 0.3822,
 'eval_samples_per_second': 259.01,
 'eval_steps_per_second': 10.465,
 'epoch': 8.0}

##  Оценим качество на отложенной выборке

In [53]:
classifiсation_pipeline = pipeline('text-classification', model=classifier, tokenizer=tokenizer, device=0)

Device set to use cuda:0


In [54]:
y_pred = list(map(lambda x: x['label'], classifiсation_pipeline(list(validation_set['text']))))
y_true = validation_set['category']
print(classification_report(y_true=y_true, y_pred=y_pred))

                    precision    recall  f1-score   support

     entertainment       1.00      0.56      0.71         9
         geography       0.75      0.75      0.75         8
            health       1.00      0.82      0.90        11
          politics       0.88      1.00      0.93        14
science/technology       0.92      0.96      0.94        25
            sports       1.00      0.92      0.96        12
            travel       0.67      0.80      0.73        20

          accuracy                           0.86        99
         macro avg       0.89      0.83      0.85        99
      weighted avg       0.88      0.86      0.86        99



In [55]:
y_pred = list(map(lambda x: x['label'], classifiсation_pipeline(list(test_set['text']))))
y_true = test_set['category']
print(classification_report(y_true=y_true, y_pred=y_pred))

                    precision    recall  f1-score   support

     entertainment       1.00      0.47      0.64        19
         geography       0.88      0.88      0.88        17
            health       0.95      0.95      0.95        22
          politics       1.00      0.93      0.97        30
science/technology       0.86      0.94      0.90        51
            sports       0.89      0.96      0.92        25
            travel       0.84      0.95      0.89        40

          accuracy                           0.90       204
         macro avg       0.92      0.87      0.88       204
      weighted avg       0.91      0.90      0.89       204

